# NSQIP Reproducibility Project
### Attempted Reproduction of "Risk Analysis and Stratification of Surgical Morbidity after Immediate Breast Reconstruction" (Fischer et al., JACS 2013)
### DOI: 10.1016/j.jamcollsurg.2013.07.004

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import ttest_ind, chi2_contingency, fisher_exact
import statsmodels.api as sm

## *Excerpt from Methods (Fischer et al.)*

### Database and analytic cohort

The 2005 to 2011 ACS-NSQIP databases were accessed on December 1, 2012 and queried to identify all female patients undergoing IBR. Per NSQIP protocol, 240 Health Insurance Portability and Accountability Act (HIPAA)-compliant variables were collected for each encounter. Deidentified patient information is freely available to all institutional members who comply with the ACS-NSQIP Data Use Agreement. The Data Use Agreement implements the protections afforded by the Health Insurance Portability and Accountability Act of 1996. Institutional Review Board / Human Investigation Committee exemption was approved by our institutions. 

Variables included patient demographic information, preoperative comorbidities and risk factors, perioperative laboratory results, information related to intraoperative proceedings and complications, as well as postoperative morbidity and mortality data for the subsequent 30-day period. Audits conducted through 2010 showed a 1.8% disagreement rate for program variables.

Women undergoing IBR procedures were identified using 2010 Current Procedural Terminology (CPT) codes for concurrent mastectomy and reconstruction: mastectomy included partial mastectomy with (19102) and without (19101) axillary lymphadenectomy, simple mastectomy (19103), subcutaneous mastectomy (19104), modified radical mastectomy (19107), and radical mastectomy (19105, 19106); implant-based reconstructions included immediate implant (19340) and tissue expander placement (19357); autologous reconstructions included the latissimus dorsi (LD) flap (19361), pedicled transverse rectus abdominus myocutaneous (pTRAM) (including microsurgical supercharging) (19367, 19368, 19369), and free transverse rectus abdominus myocutaneous (fTRAM) (19364).


Implant-based and autologous reconstructions were defined as described above. We considered all latissimus dorsi flap reconstructions to be autologous reconstructions, whether or not tissue expanders or implants were used to augment autologous tissue. For the purposes of this study, we considered the autologous component to be a more significant aspect of the reconstruction.

#### **1. Start by accessing 2005-2011 NSQIP data.**

In [2]:
#divided by AdmYR {2005, 2006}
df_05_06 = pd.read_csv("NSQIP_data/ACS_NSQIP_PUF_05_06_vr1.txt", sep='\t', na_values=["NULL", -99], low_memory=False, encoding='latin1')
df_07 = pd.read_csv("NSQIP_data/ACS_NSQIP_PUF07_TXT.txt", sep='\t', na_values=["NULL", -99], low_memory=False, encoding='latin1')
df_08 = pd.read_csv("NSQIP_data/ACS_NSQIP_PUF08_TXT.txt", sep='\t', na_values=["NULL", -99], low_memory=False, encoding='latin1')
df_09 = pd.read_csv("NSQIP_data/ACS_NSQIP_PUF09_TXT.txt", sep='\t', na_values=["NULL", -99], low_memory=False, encoding='latin1')
df_10 = pd.read_csv("NSQIP_data/ACS_NSQIP_PUF10_TXT.txt", sep='\t', na_values=["NULL", -99], low_memory=False, encoding='latin1')
df_11 = pd.read_csv("NSQIP_data/ACS_NSQIP_PUF11_TXT.txt", sep='\t', na_values=["NULL", -99], low_memory=False, encoding='latin1')

#### **2. Identify IBR patients by presence of both mastectomy and reconstruction CPT codes.**

In [3]:
datasets = [df_05_06, df_07, df_08, df_09, df_10, df_11]
cpt_columns = [
    "CPT", "OTHERCPT1", "OTHERCPT2", "OTHERCPT3", "OTHERCPT4", "OTHERCPT5",
    "OTHERCPT6", "OTHERCPT7", "OTHERCPT8", "OTHERCPT9", "OTHERCPT10",
    "CONCPT1", "CONCPT2", "CONCPT3", "CONCPT4", "CONCPT5", "CONCPT6",
    "CONCPT7", "CONCPT8", "CONCPT9", "CONCPT10"
]


# Function to clean CPT codes
def clean_cpt(code):
    if pd.isnull(code):
        return ''
    code_str = str(code).strip().split('-')[0].split('.')[0]
    return code_str  

# Apply cleaning to all CPT columns in each dataset
for df in datasets:
    for col in cpt_columns:
        df[col] = df[col].apply(clean_cpt)

#### Reproduction Uncertainty

*Some of the provided mastectomy codes are do not link to the stated procedures. Using these codes results in 39 cases.*

| CPT Code    | Stated Procedure | Actual Procedure |
| -------- | ------- | ------- |
| 19101 | Partial mastectomy without axillary lymphadenectomy| Breast biopsy; open, incisional |
| 19102 | Partial mastectomy with axillary lymphadenectomy | Breast biopsy; percutaneous, needle core, with image guidance |
| 19103 | Simple mastectomy | Breast biopsy; percutaneous, vacuum-assisted or rotating biopsy device, with image guidance |
| 19104 | Subcutaneous mastectomy | Invalid? |
| 19105 | Radical mastectomy| Cryosurgical ablation of fibroadenoma |
| 19106 | Radical mastectomy| Invalid? |
| 19107 | Modified radical mastectomy | Ivalid? |
| 19340 | Immediate implant placement | Immediate implant placement (Correct) |
| 19357 | Immediate TE placement | Immediate TE placement (Correct) |
| 19361 | Latissimus dorsi recon | Latissimus dorsi reconstruction (Correct) |
| 19367 | pTRAM | pTRAM (Correct) |
| 19368 | pTRAM | pTRAM, with supercharging (Correct) |
| 19369 | pTRAM | pTRAM, bipedicled (Correct) |
| 19364 | fTRAM | Breast reconstruction with free flap (Does include fTRAM) |

<br />

**Decision**: Will use found CPT codes of described mastectomy procedures. These codes changed in 2007 so will use both pre-2007 and post-2007 codes. Will use described and validated reconstruction CPT codes (above). 

| Validated Mastectomy CPT Codes   |  Procedure | 
| -------- | ------- | 
| 19160, 19301 | Mastectomy, partial |
| 19162, 19302 | Mastectomy, partial with axillary lymphadenopathy |
| 19180, 19303 | Mastectomy, simple, complete |
| 19182, 19304 | Mastectomy, subcutaneous |
| 19200, 19305 | Mastectomy, radical |
| 19220, 19306 | Mastectomy, radical |
| 19240, 19307 | Mastectomy, modified radical |


In [4]:
mastectomy_codes = ['19160', '19162', '19180', '19182', '19200', '19220', '19240', 
                           '19301', '19302', '19303', '19304', '19305', '19306', '19307'] #validated
#mastectomy_codes = ['19102', '19101', '19103', '19104', '19107', '19105', '19106'] #from article

implant_codes = ['19340', '19357']
autologous_codes = ['19361', '19367', '19368', '19369', '19364']

cpt_columns = [
    "CPT", "OTHERCPT1", "OTHERCPT2", "OTHERCPT3", "OTHERCPT4", "OTHERCPT5",
    "OTHERCPT6", "OTHERCPT7", "OTHERCPT8", "OTHERCPT9", "OTHERCPT10",
    "CONCPT1", "CONCPT2", "CONCPT3", "CONCPT4", "CONCPT5", "CONCPT6",
    "CONCPT7", "CONCPT8", "CONCPT9", "CONCPT10"
]

datasets = [df_05_06, df_07, df_08, df_09, df_10, df_11]
filtered_datasets = []

for df in datasets:
    
    mask_mastectomy = df[cpt_columns].isin(mastectomy_codes).any(axis=1)
    mask_implant = df[cpt_columns].isin(implant_codes).any(axis=1)
    mask_autologous = df[cpt_columns].isin(autologous_codes).any(axis=1)
    
    #Must have a mastectomy code and either an implant or autologous code in any of the CPT columns.
    mask = mask_mastectomy & (mask_implant | mask_autologous)

    filtered_df = df[ mask ].copy()
    
    filtered_df['RECON_TYPE'] = np.where(mask_autologous[mask], "Autologous", 
                                         np.where(mask_implant[mask], "Implant", np.nan)) #autologous takes priority if both
    
    filtered_datasets.append(filtered_df)

df_combined = pd.concat(filtered_datasets, axis=0)

#### **3. Filter by female sex.**

In [5]:
df_combined = df_combined[df_combined.SEX == "female"]
print(f"There are {df_combined.shape[0]} female, IBR cases.")

There are 18558 female, IBR cases.


## *Excerpt from Methods*
### Outcome

Our primary outcome was surgical complications, which included prosthetic or flap loss, unplanned return to the operating room, deep wound infection, superficial surgical site infections, and wound dehiscence. Surgical complications were treated as a dichotomous variable (none vs 1 or more). Information regarding severity was not available. All complications were identified within 30 days of the reconstructive procedure.

#### **1. Define dichotomous 'Surgical complication' variable.**

| Outcome Variable   |  NSQIP Column(s) | 
| -------- | ------- | 
| Prosthetic or flap loss | OTHGRAFL |
| Unplanned return to OR | RETURNOR, REOPERATION |
| Deep wound infection | WNDINFD |
| Superficial surgical site infection | SUPINFEC |
| Wound dehiscence | DEHIS |

#### Reproduction Uncertainty
*There are two return to operating room variables. RETURNOR is present in all years and includes both planned and unplanned return to the operating room. REOPERATION was added in 2011 to include only unplanned return to the operating room. This variable is NOT present in 2005-2010.*

**Decision**: Though Methods describe specifically unplanned return to the operating room, will continue with RETURNOR so that cases prior to 2011 can still be included.

In [6]:
complications = ["OTHGRAFL", "RETURNOR", "WNDINFD", "SUPINFEC", "DEHIS"]
for comp in complications:
    assert df_combined[comp].isna().sum() == 0

df_combined[complications] = df_combined[complications].replace("No Complication", "No")

df_combined["Complication"] = df_combined[complications].apply(
    lambda row: "Yes" if (row != "No").any() else "No",
    axis=1
)

print(f"Out of {df_combined.shape[0]} total cases, there are {sum(df_combined.Complication == 'Yes')} cases with surgical complications")

Out of 18558 total cases, there are 1859 cases with surgical complications


/tmp/ipykernel_2048048/4218336484.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_combined["Complication"] = df_combined[complications].apply(


## *Excerpt from Methods*
### Independent variables

Variables indicating patient demographics, comorbidities, and perioperative risk factors were selected for analysis. These included baseline health characteristics, past medical and surgical history, preoperative laboratory values, and American Society of Anesthesiologists (ASA) physical status. The full list and definitions of NSQIP program variables can be found on the ACS-NSQIP website (http://site.acsnsqip.org/). 

#### **1. Clean demographics variables**

#### Reproduction Uncertainty
*Methods does not list which demographic variables were considered.*

**Decision**: Demographic variables listed throughout the Results will be cleaned.

| Demographic Variable   |  NSQIP Column(s) | 
| -------- | ------- |
| Race | RACE, RACE_NEW |
| Year | OperYR |
| Age | Age |
| Inpatient status | INOUT |

<br/>


#### Reproduction Uncertainty
*For Race, the RACE variable was used prior to 2008, when it was split into RACE_NEW and ETHNICITY_HISPANIC to comply with updated guidelines. The Results show the options, "White", "Black", "Hispanic", and "Other". However, both RACE and RACE_NEW have more specific variables, like "Asian or Pacific Islander" or "Hispanic, Black". Methods do not specify how to divide Mixed indivdiuals, for example are "Hispanic, Black" patients labeled as "Hispanic" or "Black"*

**Decision**: Will use the RACE and RACE_NEW variables, converting them into four categories ("White", "Black", "Hispanic", "Other"). For "Hispanic, Black", will label as "Hispanic"

In [7]:
df_combined['RACE_COMBINED'] = df_combined['RACE_NEW'].fillna(df_combined['RACE'])


df_combined.loc[df_combined["RACE_COMBINED"] == "Black, Not of Hispanic Origin", "RACE_COMBINED"] = "Black"
df_combined.loc[df_combined["RACE_COMBINED"] == "Black or African American", "RACE_COMBINED"] = "Black"

df_combined.loc[df_combined["RACE_COMBINED"] == "White, Not of Hispanic Origin", "RACE_COMBINED"] = "White"

df_combined.loc[df_combined["RACE_COMBINED"] == "Hispanic, Black", "RACE_COMBINED"] = "Hispanic"
df_combined.loc[df_combined["RACE_COMBINED"] == "Hispanic, White", "RACE_COMBINED"] = "Hispanic"
df_combined.loc[df_combined["RACE_COMBINED"] == "Hispanic, Color Unknown", "RACE_COMBINED"] = "Hispanic"

df_combined.loc[df_combined["RACE_COMBINED"] == "Asian", "RACE_COMBINED"] = "Other"
df_combined.loc[df_combined["RACE_COMBINED"] == "Asian or Pacific Islander", "RACE_COMBINED"] = "Other"
df_combined.loc[df_combined["RACE_COMBINED"] == "Native Hawaiian or Pacific Islander", "RACE_COMBINED"] = "Other"
df_combined.loc[df_combined["RACE_COMBINED"] == "American Indian or Alaska Native", "RACE_COMBINED"] = "Other"

df_combined.loc[df_combined["RACE_COMBINED"] == "Unknown", "RACE_COMBINED"] = np.nan

/tmp/ipykernel_2048048/1195045764.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_combined['RACE_COMBINED'] = df_combined['RACE_NEW'].fillna(df_combined['RACE'])


In [8]:
demographic_vars = ["RACE_COMBINED", "OperYR", "Age", "INOUT"]
print(f"There are {df_combined[demographic_vars].isna().any(axis=1).sum()} cases with missing data.")

There are 1760 cases with missing data.


#### Reproduction Uncertainty
*Methods does not describe what to do with cases with missing independent variables.*

**Decision**: Cases with missing variables will be excluded.

In [9]:
df_combined = df_combined.dropna(subset=demographic_vars)

#### **1a. Create ranges for age and year**

In the Results, age includes ranges <45, 45-64, and >65. Year is divided between 2005-2008 and 2009-2011

In [10]:
#age
assert(df_combined.Age.isna().sum() == 0)

df_combined = df_combined.copy()

df_combined["Age"] = df_combined["Age"].replace("90+", 90)
df_combined["Age"] = pd.to_numeric(df_combined["Age"], errors="coerce")

age_bins = [0, 45, 65, float('inf')]
age_labels = ["<45", "45–64", "65+"]

df_combined["Age_range"] = pd.cut(df_combined["Age"], bins=age_bins, labels=age_labels, right=False)

In [11]:
#year
assert(df_combined.OperYR.isna().sum() == 0)

yr_bins = [2005, 2009, 2012]
yr_labels = ["2005-2008", "2009-2011"]

df_combined["OperYR_range"] = pd.cut(df_combined["OperYR"], bins=yr_bins, labels=yr_labels, right=False)

In [12]:
df_combined[["Age", "Age_range", "OperYR", "OperYR_range"]]

,Age,Age_range,OperYR,OperYR_range
3627,44,<45,2005,2005-2008
5357,44,<45,2005,2005-2008
7481,37,<45,2005,2005-2008
7484,48,45–64,2005,2005-2008
7485,38,<45,2005,2005-2008
...,...,...,...,...
441561,31,<45,2011,2009-2011
441613,64,45–64,2011,2009-2011
441616,26,<45,2011,2009-2011
442011,29,<45,2011,2009-2011


## *Excerpt from Methods*
### Independent variables (continued)

The World Health Organization definition of obesity was used to classify patients with a body mass index (BMI) <30 kg/m2 as nonobese and those with BMI >30 kg/m2 as obese. Patients were defined as follows: nonobese (BMI <30 kg/m2), class I obese (BMI = 30 to 34.9 kg/m2), class II obese (BMI = 34.9 to 39.9 kg/m2), and class III obese (BMI >= 40 kg/m2).  Comorbidities were examined individually, by system, and in aggregate. System-based definitions were as follows: cardiovascular included a history of hypertension, angina, congestive heart failure, myocardial infarction, percutaneous coronary intervention, cardiac surgery, rest pain, or peripheral vascular disease; pulmonary included recent dyspnea, COPD, recent pneumonia, or a recent need for ventilator-assisted respiration; neurologic included paralysis, recent coma, recent delirium, cerebrovascular accident, or transient ischemic attack; and renal included preoperative renal insufficiency and dialysis. In aggregate, overall comorbidity burden was defined as no comorbidity, 1 comorbidity, and 2 or more comorbidities.

#### **1. Define system-based comorbidities based on provided definitions**

- **Cardiovascular**

| Comorbidity Variable   |  NSQIP Column(s) | 
| --------- | --------- |
| Hypertension | HYPERMED |
| Angina | HXANGINA |
| Congestive heart failure | HXCHF |
| Myocardial infarction | HXMI |
| Percutaneous coronary intervention | PRVPCI |
| Cardiac surgery | PRVPCS |
| Rest pain | RESTPAIN |
| Periphearl vascular disease | HXPVD |


- **Pulmonary**

| Comorbidity Variable   |  NSQIP Column(s) | 
| --------- | --------- |
| Dyspnea | DYSPNEA |
| COPD | HXCOPD |
| Recent pneumonia | CPNEUMON |
| Ventilator-assisted respiration | VENTILAT |

- **Neurologic**

| Comorbidity Variable   |  NSQIP Column(s) | 
| --------- | --------- |
| Paralysis | HEMI, QUAD |
| Recent coma | COMA |
| Recent delirium | IMPSENS |
| Recent cerebrovascular accident | CVA, CVANO |
| Recent transient ischemic attack | HXTIA |

- **Renal**

| Comorbidity Variable   |  NSQIP Column(s) | 
| --------- | --------- |
| Preoperative renal insufficiency | RENAFAIL |
| Dialysis | DIALYSIS |


In [13]:
sys_comorbid = ["HYPERMED", "HXANGINA", "HXCHF", "HXMI", "PRVPCI", 
                "PRVPCS", "RESTPAIN", "HXPVD",
               "DYSPNEA", "HXCOPD", "CPNEUMON", "VENTILAT", 
               "HEMI", "QUAD", "COMA", "IMPSENS", "CVA", "CVANO", "HXTIA",
               "RENAFAIL", "DIALYSIS"]

print(f"There are {df_combined[sys_comorbid].isna().any(axis=1).sum()} cases with missing data.")

There are 2045 cases with missing data.


#### Reproduction Uncertainty
*There are multiple cases with missing comorbidity data.*

**Decision**: Will drop cases with any missing data.

In [14]:
df_combined = df_combined.dropna(subset=sys_comorbid)
df_combined = df_combined.copy()

In [15]:
#Add cardio comorbidities
cardio_comorbid = ["HYPERMED", "HXANGINA", "HXCHF", "HXMI", "PRVPCI", 
                "PRVPCS", "RESTPAIN", "HXPVD"]
df_combined["CARDIO_COMORBID"] = df_combined[cardio_comorbid].eq("Yes").any(axis=1)
df_combined["CARDIO_COMORBID"] = df_combined["CARDIO_COMORBID"].map({True: "Yes", False: "No"})

#Add pulm comorbidities
pulm_comorbid = ["DYSPNEA", "HXCOPD", "CPNEUMON", "VENTILAT" ]
df_combined["PULM_COMORBID"] = df_combined[pulm_comorbid].eq("Yes").any(axis=1)
df_combined["PULM_COMORBID"] = df_combined["PULM_COMORBID"].map({True: "Yes", False: "No"})

#Add neuro comorbidities
neuro_comorbid = ["HEMI", "QUAD", "COMA", "IMPSENS", "CVA", "CVANO", "HXTIA"]
df_combined["NEURO_COMORBID"] = df_combined[neuro_comorbid].eq("Yes").any(axis=1)
df_combined["NEURO_COMORBID"] = df_combined["NEURO_COMORBID"].map({True: "Yes", False: "No"})


#Add renal comorbidities
renal_comorbid = ["RENAFAIL", "DIALYSIS"]
df_combined["RENAL_COMORBID"] = df_combined[renal_comorbid].eq("Yes").any(axis=1)
df_combined["RENAL_COMORBID"] = df_combined["RENAL_COMORBID"].map({True: "Yes", False: "No"})

#### **2. Clean other comorbidity variables.**

#### Reproduction Uncertainty
*Methods did not list the other variables used for comorbidities.*

**Decision**: Will use the comorbidities listed throughout the Results section.

| Comorbidity Variable   |  NSQIP Column(s) | 
| --------- | --------- |
| BMI | Calculate from HEIGHT, WEIGHT |
| Obesity | Define based on calculated BMI |
| Diabetes | DIABETES |
| American Society of Anesthesiologists | ASACLAS |
| Functional status | FNSTATUS1, FNSTATUS2 |
| Smoking | SMOKE |
| Alchohol use | ETOH |
| Radiation within 90d | RADIO |
| Chemotherapy within 30d | CHEMO |
| Previous oepration wtihin 30d | PrOper30 |
| Currently on steroids | STEROID |


<br/>

#### Reproduction Uncertainty
*There are two functional status variables, FNSTATUS1 describes status prior to current illness and FNSTATUS2 describes status prior to surgery.*

**Decision**: Will use FNSTATUS2 to reflect functional status prior to surgery

In [16]:
ind_comorbid = ["HEIGHT", "WEIGHT", "DIABETES", "ASACLAS", "FNSTATUS2", "SMOKE", 
                "ETOH", "RADIO", "CHEMO", "PrOper30", "STEROID"]
df_combined[ df_combined.ASACLAS == "None assigned" ] = np.nan

print(f"There are {df_combined[ind_comorbid].isna().any(axis=1).sum()} cases with missing data.")

There are 767 cases with missing data.


#### Reproduction Uncertainty
*There are multiple cases with missing comorbidity data.*

**Decision**: Will drop cases with any missing data.

In [17]:
df_combined = df_combined.dropna(subset=ind_comorbid)

#### **2a. Calcualte BMI and define obesity classes based on provided definitions**

- < 30 : non-obese
- 30-34.9: Class I Obesity
- 40-39.9: Class II Obesity
- 40+: Class III Obesity

In [18]:
df_combined["BMI"] = (df_combined["WEIGHT"] / (df_combined["HEIGHT"] ** 2)) * 703
def classify_bmi(bmi):
    if bmi < 30:
        return "Nonobese"
    elif 30 <= bmi < 35:
        return "Class I Obese"
    elif 35 <= bmi < 40:
        return "Class II Obese"
    elif bmi >= 40:
        return "Class III Obese"

df_combined["OBESITY_CLASS"] = df_combined["BMI"].apply(classify_bmi)
df_combined["OBESITY_BINARY"] = np.where(
    df_combined['OBESITY_CLASS'] == "Nonobese",
    'No', 'Yes'
)

In [19]:
df_combined[["HEIGHT", "WEIGHT", "BMI", "OBESITY_CLASS", "OBESITY_BINARY"]]

,HEIGHT,WEIGHT,BMI,OBESITY_CLASS,OBESITY_BINARY
34060,65.0,140.0,23.294675,Nonobese,No
34163,62.0,88.0,16.093652,Nonobese,No
34497,59.0,97.0,19.589486,Nonobese,No
34817,70.0,176.0,25.250612,Nonobese,No
36548,69.0,188.0,27.759714,Nonobese,No
...,...,...,...,...,...
440520,56.0,137.0,30.711416,Class I Obese,Yes
441435,62.0,140.0,25.603538,Nonobese,No
441561,65.0,105.0,17.471006,Nonobese,No
442011,66.0,157.0,25.337695,Nonobese,No


#### **2b. Categorize ASA Class**

In the Results, ASA Class is divided into ASA Class 1-2 and ASA Class 3+

In [20]:
df_combined['ASACLAS_BIN'] = np.where(
    df_combined['ASACLAS'].str.startswith(('1', '2')),
    '1-2',
    np.where(df_combined['ASACLAS'].str.startswith(('3', '4', '5')), '3+', np.nan)
)

df_combined["ASACLAS_BINARY"] = np.where(
    df_combined['ASACLAS_BIN'] == "3+",
    'Yes', 'No'
)

df_combined[['ASACLAS', 'ASACLAS_BIN', 'ASACLAS_BINARY']]

,ASACLAS,ASACLAS_BIN,ASACLAS_BINARY
34060,2-Mild Disturb,1-2,No
34163,2-Mild Disturb,1-2,No
34497,2-Mild Disturb,1-2,No
34817,2-Mild Disturb,1-2,No
36548,2-Mild Disturb,1-2,No
...,...,...,...
440520,2-Mild Disturb,1-2,No
441435,2-Mild Disturb,1-2,No
441561,2-Mild Disturb,1-2,No
442011,2-Mild Disturb,1-2,No


#### **2c. Clean Diabetes and Functional Status**

Convert these columns to binary variables. For example the DIABETES column currently has "No", "Oral", "Non-Insulin", and "Insulin" categories to describe each patient. Need to convert to Yes / No. 

In [21]:
#DIABETES_BINARY will be yes if Oral, Non-Insulin, or Insulin. 
df_combined["DIABETES_BINARY"] = np.where(
    df_combined['DIABETES'] == "NO",
    'No', 'Yes'
)

df_combined[["DIABETES", "DIABETES_BINARY"]][117:127]

,DIABETES,DIABETES_BINARY
69214,NO,No
69281,NO,No
69702,NO,No
70243,INSULIN,Yes
70286,NO,No
70439,NO,No
70579,NO,No
70644,ORAL,Yes
71065,NO,No
71128,NO,No


In [22]:
#FUNDEPENDENT will be yes if patient is partially or totally dependent prior to surgery
df_combined["FUNDEPENDENT"] = np.where(
    df_combined['FNSTATUS2'] == "Independent",
    'No', 'Yes'
)

df_combined[["FNSTATUS2", "FUNDEPENDENT"]][110:120]

,FNSTATUS2,FUNDEPENDENT
67860,Independent,No
67949,Independent,No
68023,Independent,No
68431,Independent,No
68554,Independent,No
68909,Independent,No
69164,Partially Dependent,Yes
69214,Independent,No
69281,Independent,No
69702,Independent,No


#### **3. Define aggregate comorbidity burden**

As stated earlier, Methods says, "In aggregate, overall comorbidity burden was defined as no comorbidity, 1 comorbidity, and 2 or more comorbidities."

In [33]:
all_comorbid = ["CARDIO_COMORBID", "PULM_COMORBID", "NEURO_COMORBID", "RENAL_COMORBID",
               "OBESITY_BINARY", "DIABETES_BINARY", "ASACLAS_BINARY", "FUNDEPENDENT", "SMOKE", 
                "ETOH", "RADIO", "CHEMO", "PrOper30", "STEROID"]

assert df_combined[all_comorbid].isna().any(axis=1).sum() == 0

#Add number of comorbidities: None, One, Two or More
df_combined["NUM_COMORBID"] = df_combined[all_comorbid].eq("Yes").sum(axis=1)
df_combined["NUM_COMORBID"] = df_combined["NUM_COMORBID"].apply(
    lambda x: "None" if x == 0 else ("One" if x == 1 else "Two or More")
)


In [34]:
all_comorbid.append("NUM_COMORBID")
df_combined[all_comorbid]

,CARDIO_COMORBID,PULM_COMORBID,NEURO_COMORBID,RENAL_COMORBID,OBESITY_BINARY,DIABETES_BINARY,ASACLAS_BINARY,FUNDEPENDENT,SMOKE,ETOH,RADIO,CHEMO,PrOper30,STEROID,NUM_COMORBID
34060,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,One
34163,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,One
34497,Yes,No,No,No,No,No,No,No,Yes,No,No,No,No,No,Two or More
34817,No,No,No,No,No,No,No,No,No,No,No,No,No,No,None
36548,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,One
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440520,Yes,No,Yes,No,Yes,No,No,No,No,No,No,No,No,No,Two or More
441435,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,One
441561,No,No,No,No,No,No,No,No,No,No,No,No,No,No,None
442011,No,No,No,No,No,No,No,No,No,No,No,No,No,No,None


## *Excerpt from Methods*
### Independent variables (continued)
Implant-based and autologous reconstructions were defined as described above. We considered all latissimus dorsi flap reconstructions to be autologous reconstructions, whether or not tissue expanders or implants were used to augment autologous tissue. For the purposes of this study, we considered the autologous component to be a more significant aspect of the reconstruction. Bilateral reconstruction and bilateral mastectomy were defined when 2 concurrent CPT codes for the procedure were present. Use of acellular dermal matrix (ADM) was defined using CPT codes 15170, 15330, 15331, 15335, 15340, and 15341.

#### **1. Define implant-based vs. autologous reconstruction**
This was previously done when data was loaded. Note that if patients had both implant and autologous reconstruction, they were defined as autologous as inferred from the Methods. 

In [37]:
df_combined.RECON_TYPE.value_counts()

RECON_TYPE
Implant       11218
Autologous     2768
Name: count, dtype: int64

#### **2. Define unilateral vs. bilateral procedures**


#### Reproduction Uncertainty
*The Methods describe defining bilateral reconstruction and mastectomy which is reflected in Results Table 2. However, in Results Table 1, there is a variable "Procedures; Unilateral or Bilateral."*

**Decision**: Will keep bilateral mastectomy and reconstruction separate. 

#### Reproduction Uncertainty
*The Methods state, "bilateral mastectomy were defined when 2 concurrent CPT codes for the procedure were present." However, what if a case has a primary CPT code as mastectomy and one concurrent CPT code as a second mastectomy. Here, the patient had a bilateral mastectomy but only one of the CPT codes is technically listed as a "concurrent CPT code."*

**Decision**: Will search all CPT codes for laterality definitions rather than just concurrent CPT codes

In [38]:
#mastectomy codes
mastectomy_codes = ['19160', '19162', '19180', '19182', '19200', '19220', '19240', 
                           '19301', '19302', '19303', '19304', '19305', '19306', '19307']

mastectomy_present = df_combined[cpt_columns].isin(mastectomy_codes)
mastectomy_counts = mastectomy_present.sum(axis=1)

# Assign "Bilateral" if count >=2, else "Unilateral"
df_combined["Mastectomy_Laterality"] = np.where(
    mastectomy_counts >= 2,
    "Bilateral",
    "Unilateral"
)

df_combined["Mastectomy_Laterality"].value_counts()

Mastectomy_Laterality
Unilateral    8936
Bilateral     5050
Name: count, dtype: int64

In [39]:
#reconstruction codes including both implant and autologous
reconstruction_codes = ['19340', '19357', '19361', '19367', '19368', '19369', '19364']

reconstruction_present = df_combined[cpt_columns].isin(reconstruction_codes)
reconstruction_counts = reconstruction_present.sum(axis=1)

# Assign "Bilateral" if count >=2, else "Unilateral"
df_combined["Reconstruction_Laterality"] = np.where(
    reconstruction_counts >= 2,
    "Bilateral",
    "Unilateral"
)

df_combined["Reconstruction_Laterality"].value_counts()

Reconstruction_Laterality
Unilateral    9351
Bilateral     4635
Name: count, dtype: int64

#### **2. Define ADM usage**


In [40]:
adm_codes = ['15170', '15330', '15331', '15335', '15340', '15341']

df_combined["ADM_USAGE"] = df_combined[cpt_columns].apply(
    lambda row: "Yes" if any(code in adm_codes for code in row) else "No", axis=1
)

df_combined[["ADM_USAGE"]].value_counts()

ADM_USAGE
No           11682
Yes           2304
Name: count, dtype: int64

# *Excerpt from Methods*
## Internal validation and statistical analysis

Two-thirds of the study sample was randomly selected as our “model cohort.” Independent predictors of surgical morbidity were identified and a risk scoring system was created using the model cohort, while the remaining one-third, the “validation cohort” was used to assess the validity of our predictive model.16 Bivariate analysis of independent variables by our outcome of interest was performed using Pearson chi-square or Fisher’s exact tests for categorical variables and Wilcoxon rank-sum or Mann-Whitney tests for continuous variables within the “model cohort.” Preoperative and intraoperative variables with a p < 0.10 on univariate analysis in the “model cohort” were included in the multivariate logistic regression analysis. Significant risk factors derived from the regression analysis were weighted using rounded odds ratios to create a composite risk score for each patient, called the IBR risk assessment scale (IBRRAS) (range of 0 to 9). Patients were then stratified by assigned risk score and categorized as low (0 to 2), intermediate (3 to 4), high (5 to 7), or very high (8 to 9) risk. To assess the accuracy of the IBRRAS, we applied the IBRRAS to the remaining one-third of the study group, or the “validation cohort.” A post-hoc comparison of surgical complication rates for each defined risk level (low, intermediate, high, and very high) in the “model cohort” and “validation cohort” was performed to assess the internal validity of the model. All tests were 2-tailed, with statistical significance defined as p < 0.05.

#### Reproduction Uncertainty
*To recreate the IBRRAS, the model cohort would need to be determined with specific preoperative and intraoperative variables. These are not specified in the methods. Further, in this setting the Wilcoxon rank-sum and Mann-Whitney test would be referring to the same statistical test? It is unclear why both are mentioned. Finally, converting "rounded odds ratios" into a "composite risk score" does not have a clear mathematical formula.*

**Decision**: Unable to recreate the IBBRAS